In [37]:
from Bio import SeqIO
import phylopandas as ph
import numpy as np
import pandas as pd
from Bio.SwissProt import KeyWList

import re
import networkx as nx
import obonet


VECTOR_LEN=343

In [2]:
records = list(SeqIO.parse("uniref.fasta", "fasta"))

In [3]:
df = ph.read_fasta('uniref.fasta')

1 A, G, V

2 I, L, F, P

3 Y, M, T, S

4 H, N, Q, W

5 R, K

6 D, E

7 C


In [4]:
triads={1:['A', 'G', 'V'],2:['I', 'L', 'F', 'P'], 3:['Y', 'M', 'T', 'S'],
        4:['H', 'N', 'Q', 'W','B','Z'], 5:[ 'R', 'K','O'], 6:['D', 'E'], 7:['C','U']}
def letter_to_number(item):
    if item in triads[1]:
        return 1
    elif item in triads[2]:
        return 2
    elif item in triads[3]:
        return 3
    elif item in triads[4]:
        return 4
    elif item in triads[5]:
        return 5
    elif item in triads[6]:
        return 6
    elif item in triads[7]:
        return 7
    else: 
        return 0

In [5]:
seqs_numbers=[]
check=0
count=0
for i, seq in enumerate(df.sequence):
    seq_numbers=""
    for j,amino_acid in enumerate(seq):
        let=letter_to_number(amino_acid)
        if(let!=0):
            seq_numbers+=(str(let))
        else:
            count+=1
#             print("NEEEEEEEEEEEEEEEEEEEEEEEE"+str(i))
            check=1
            df.drop([i],inplace=True)
            break
    if check==0:        
        seqs_numbers.append(seq_numbers)    
    else:
        check=0
print(count)   

874


In [6]:
df["triad_method"]=seqs_numbers

In [7]:
df.shape

(183522, 6)

In [8]:
three_to_three={}
f=1
for i in range(1,8):
    for j in range(1,8):
        for k in range(1,8):
            n=str(k)+str(j)+str(i)
            three_to_three[n]=f
            f+=1

In [9]:
def create_vector(seq,i):
    vect = np.zeros(VECTOR_LEN+1)
    ln = len(seq)-2
    for i in range(0, ln):
        three = seq[i:i+3]
        try:
            vect[three_to_three[three]]+=1
        except:
            print(i)
    return vect    

In [10]:
len(three_to_three)

343

In [11]:
vectors_of_tree=[]
for i, seq in enumerate(df['triad_method']):    
    vectors_of_tree.append(create_vector(seq,i))    

In [12]:
len(vectors_of_tree)

183522

In [13]:
df['vectors']=vectors_of_tree

In [14]:
import re
arr_i=[]
for i,ids in enumerate(df.id):
    arr_i.append(re.sub('UniRef50_', '', ids))

In [15]:
len(arr_i)

183522

In [16]:
df.id

0         UniRef50_Q8WZ42
1         UniRef50_G4SLH0
2         UniRef50_Q9I7U4
3         UniRef50_Q8WXI7
4         UniRef50_Q09165
               ...       
184391    UniRef50_P85444
184392    UniRef50_P83092
184393    UniRef50_P80464
184394    UniRef50_P12797
184395    UniRef50_B3EWL7
Name: id, Length: 183522, dtype: object

In [17]:
df.id=arr_i

In [18]:
data = pd.read_csv('uniprot-reviewed_yes.tab',sep = '\t')

In [19]:
data=data[data.Entry.isin(df.id)]

In [20]:
data.dropna(inplace=True)

In [21]:
data.reset_index(inplace=True)

In [22]:
id_arr_not_final = []
term_arr_not_final = []

tmp_count = -1
termS = {}
arr_of_terms=[]
for i,line1 in enumerate(data['Gene ontology (molecular function)']):
    termList = re.findall(r'(GO:[0-9]+)', line1)
    try:
        tmp_l = []
        for i,elem in enumerate(termList):
            tmp_l.append(elem[3:])
        termS[data.Entry[i]]=tmp_l
        arr_of_terms.append(tmp_l)
    except Exception as e:
        print(e)
        print(termList)

In [23]:
data['Termin']=arr_of_terms

In [24]:
data.drop(columns=['index'], inplace=True)

In [25]:
df.reset_index(inplace=True)

In [26]:
df.drop(columns=['index'], inplace=True)

In [27]:
data=data.sort_values(by=['Entry'])
df= df.sort_values(by='id',  na_position='first')

In [28]:
lala={}
for index, row in df.iterrows():
    lala[row['id']]=row['vectors'].tolist()

In [29]:
elem=[]
for i in data.Entry:
    try:
        elem.append(lala[i])
#         print('ok')
    except Exception as e:
        print(i)

In [30]:
data['vector']=elem

In [31]:
# data

In [32]:
# data.to_csv(r'data.csv',index=False)

In [33]:
# data=pd.read_csv('data.csv')

In [34]:
# data

In [35]:
graph = obonet.read_obo('http://purl.obolibrary.org/obo/go/go-basic.obo')

# Number of nodes
print(len(graph))
print(graph.number_of_edges())
print(networkx.is_directed_acyclic_graph(graph))

44508
91124
True


In [38]:
# import networkx as nx
# graph = obonet.read_obo('http://purl.obolibrary.org/obo/go/go-basic.obo')

gProc = nx.DiGraph()
gComp = nx.DiGraph()
gFunc = nx.DiGraph()
for i, node in enumerate(graph.nodes):
    elem = graph.nodes[node]
    
    idName=node[3:]
    if idName == 'er_in_t':
        print('here')
    else:    
        if elem.get("namespace")=='cellular_component':
            gComp.add_node(idName)
        if elem.get("namespace")=='biological_process':
            gProc.add_node(idName)
        if elem.get("namespace")=='molecular_function':
            gFunc.add_node(idName)    
    if elem.get("is_a"):
        parent = elem.get("is_a")[0][3:]
        if elem.get("namespace")=='cellular_component':
            gComp.add_edge(idName,parent)
        if elem.get("namespace")=='biological_process':
            gProc.add_edge(idName,parent)
        if elem.get("namespace")=='molecular_function':
            gFunc.add_edge(idName,parent)

In [39]:
print(gComp.number_of_nodes())
print(gProc.number_of_nodes())
print(gFunc.number_of_nodes())
dic = {}
for i, node in enumerate(graph.nodes):
    elem = graph.nodes[node]
    if elem.get('alt_id'):
        dic[node[3:]]=elem.get('alt_id')[0][3:]

4184
29211
11113


In [40]:
gFuncTemp = gFunc.copy()
print(gFuncTemp.number_of_nodes())
leaf = [x for x in gFuncTemp.nodes() if gFuncTemp.out_degree(x)!=0 and gFuncTemp.in_degree(x)==0]
gFuncTemp.remove_nodes_from(leaf)
gFuncCut1 = gFuncTemp.copy()
print(gFuncTemp.number_of_nodes())
leafTemp = [x for x in gFuncTemp.nodes() if gFuncTemp.out_degree(x)!=0 and gFuncTemp.in_degree(x)==0]
gFuncTemp.remove_nodes_from(leafTemp)
print(gFuncTemp.number_of_nodes())

leafTemp1 = [x for x in gFuncTemp.nodes() if gFuncTemp.out_degree(x)!=0 and gFuncTemp.in_degree(x)==0]
gFuncTemp.remove_nodes_from(leafTemp1)
print(gFuncTemp.number_of_nodes())

leafTemp2 = [x for x in gFuncTemp.nodes() if gFuncTemp.out_degree(x)!=0 and gFuncTemp.in_degree(x)==0]
gFuncTemp.remove_nodes_from(leafTemp2)
print(gFuncTemp.number_of_nodes())
leafTemp3 = [x for x in gFuncTemp.nodes() if gFuncTemp.out_degree(x)!=0 and gFuncTemp.in_degree(x)==0]
gFuncTemp.remove_nodes_from(leafTemp3)
leafTemp3 = [x for x in gFuncTemp.nodes() if gFuncTemp.out_degree(x)!=0 and gFuncTemp.in_degree(x)==0]
gFuncTemp.remove_nodes_from(leafTemp3)
print(gFuncTemp.number_of_nodes())
print(gFuncTemp.nodes())
print('--------------------------------------')
term_arr=list(gFuncTemp.nodes())
term_arr.sort()
TERM_VECTOR_LEN=len(term_arr)

11113
1711
616
264
125
31
['0016740', '0005488', '0008324', '0005515', '0060089', '0036094', '0016491', '0016462', '0003676', '0038023', '0003674', '0097159', '0003824', '0016787', '0016818', '0016747', '0016788', '0005215', '0005216', '0015075', '0022857', '0005342', '0015144', '0016407', '0016741', '0022890', '0022803', '0098772', '0016772', '0016746', '0016817']
--------------------------------------


In [41]:
%%time
wrong=set()
def find_parents(id_name):
    list_descendants=[]
    try:
        list_descendants = nx.descendants(gFunc, id_name)
    except Exception as e:
        if id_name=='0044212':
            find_parents('0000976')
        else:
            wrong.update([id_name])
    return list_descendants


CPU times: user 16 µs, sys: 0 ns, total: 16 µs
Wall time: 26.2 µs


In [42]:
TERM_VECTOR_LEN

31

In [43]:
%%time
arr_vec_term=[]
max_len=0
exep=set()
for termins in data.Termin:
    vect=np.zeros(TERM_VECTOR_LEN)
    aset=set()
    for j in termins:        
#         aset.update([j])
        aset.update(find_parents(j))
    if max_len<len(aset):
        max_len=len(aset)
    for term_aset in aset:
        try:
            vect[term_arr.index(term_aset)]=1
        except Exception as e:
            exep.update(term_aset)
    arr_vec_term.append(vect)
    

CPU times: user 11.5 s, sys: 24 ms, total: 11.6 s
Wall time: 11.6 s


In [44]:
data['term_vec_non_zero']=arr_vec_term

In [45]:
data

,Entry,Gene ontology (molecular function),Termin,vector,term_vec_non_zero
95391,A0A009IHW8,NAD(P)+ nucleosidase activity [GO:0050135]; NA...,"[0050135, 0003953, 0061809]","[0.0, 0.0, 2.0, 1.0, 0.0, 3.0, 0.0, 0.0, 1.0, ...","[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
46075,A0A023IWI4,toxin activity [GO:0090729],[0090729],"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
18271,A0A023IWM6,toxin activity [GO:0090729],[0090729],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
53991,A0A023UCC1,toxin activity [GO:0090729],[0090729],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
126599,A0A023VZF1,toxin activity [GO:0090729],[0090729],"[0.0, 2.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...
97248,X5CWH9,"2 iron, 2 sulfur cluster binding [GO:0051537];...","[0051537, 0009055, 0046872]","[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, ...","[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
130698,X5HYT8,DNA binding [GO:0003677]; DNA-binding transcri...,"[0003677, 0003700]","[0.0, 20.0, 5.0, 2.0, 2.0, 1.0, 5.0, 1.0, 2.0,...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
34986,X5IWS1,ion channel inhibitor activity [GO:0008200]; t...,"[0008200, 0090729]","[0.0, 1.0, 2.0, 0.0, 0.0, 0.0, 0.0, 2.0, 1.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
110763,X5M5N0,ATP binding [GO:0005524]; chloride channel inh...,"[0005524, 0019869, 0019870, 0004674]","[0.0, 28.0, 10.0, 15.0, 6.0, 5.0, 6.0, 0.0, 24...","[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."


In [46]:
seq_file = open('seq_file_31.txt','w')
id_file = open('id_file_31.txt','w')

for index, row in data.iterrows():
    id_file.write(row['Entry'] + '\n')
    seq_file.write(str(row['vector']).strip()+ '\n')
#     term_file.write(str(row['term_vec']).strip()+ '\n')
id_file.close()  
seq_file.close()  


In [47]:
term_file = open('term_file_31.txt','w')

for line in arr_vec_term:  
    term_file.write(str(line).strip().replace('\n', '') + '\n')
term_file.close()        

In [49]:
with open('term_file_31.txt','r') as file:
    for line in file:
        print(line)
        break
        
        

[1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

